In [2]:
%env AWS_PROFILE=dev-admin
%env AWS_REGION=us-east-1
%env HF_HOME=~/.cache/huggingface
%env TOKENIZERS_PARALLELISM=fale

env: AWS_PROFILE=dev-admin
env: AWS_REGION=us-east-1
env: HF_HOME=~/.cache/huggingface
env: TOKENIZERS_PARALLELISM=fale


In [4]:
%env

{'COMMAND_MODE': 'unix2003',
 'HOME': '/Users/andrewbeiler',
 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar',
 'HOMEBREW_PREFIX': '/opt/homebrew',
 'HOMEBREW_REPOSITORY': '/opt/homebrew',
 'INFOPATH': '/opt/homebrew/share/info:',
 'LOGNAME': 'andrewbeiler',
 'MANPATH': '/opt/homebrew/share/man::',
 'MallocNanoZone': '0',
 'OLDPWD': '/',
 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined',
 'PATH': '/Users/andrewbeiler/projects/llm-data-driven-optimization/.venv/bin:/Library/Frameworks/Python.framework/Versions/3.8/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Applications/VMware Fusion.app/Contents/Public:/Library/Apple/usr/bin:/Library/Frameworks/Mono.framework/Versions/Current/Commands',
 'PWD': '/',
 'SHELL': '/bin/zsh',
 'SHLVL': '1',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.jeVzgmI8ek/Listeners',
 'TMPDIR': '/var/folders/b7/s1kskgdx4txd_4ctzl8t3xpr0000gn/T/',
 'USER': 'andrewbeiler',
 'VSCODE_AMD_ENTRYPOINT': 'vs/workbench/api/node/extensionHo

In [3]:
import sagemaker
import boto3

sagemaker_session_bucket = "sagemaker-ms-thesis-llm"
role = "arn:aws:iam::171706357329:role/service-role/SageMaker-ComputeAdmin"
boto_sess = boto3.Session(region_name="us-east-1")
sess = sagemaker.Session(boto_session=boto_sess, default_bucket=sagemaker_session_bucket)

In [4]:
def format_goat_data(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    response = f"### Answer\n{sample['output']}"
    prompt = "\n\n".join([i for i in [instruction, response] if i is not None])
    return prompt

In [5]:
import math
from datasets import load_dataset
from random import randrange
from itertools import chain
from functools import partial

from transformers import AutoTokenizer

from prompter import Prompter

from init import init_sagemaker


prompter = Prompter()
tokenizer = None

cutoff_len = 512

model_id = "meta-llama/Llama-2-7b-hf"

dataset = load_dataset("tiedong/goat", split="train")
print(f"dataset size: {len(dataset)}")
print(f"Example Datapoint {dataset[randrange(len(dataset))]}")
# print(math.ceil(len(dataset)/10))

/Users/andrewbeiler/projects/llm-data-driven-optimization/venv_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Found cached dataset json (/Users/andrewbeiler/.cache/huggingface/datasets/tiedong___json/tiedong--goat-55b7467c033a1462/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 1746300
Example Datapoint {'instruction': 'What is 279x86507440', 'output': '279 * 86507440 = 86507440 * (200 + 70 + 9) = 86507440 * 200 + 86507440 * 70 + 86507440 * 9 = 17301488000 + 6055520800 + 778566960 = 23357008800 + 778566960 = 24135575760', 'answer': '24135575760', 'input': '279 * 86507440'}


In [6]:
from random import randrange

print(format_goat_data(dataset[randrange(len(dataset))]))

### Instruction
what is 3117112959 + 335153928218

### Answer
3117112959 + 335153928218 = 338271041177


In [11]:
n_samples = 10#0000
n_shards = math.ceil(len(dataset)/n_samples)
dataset_sampled = dataset.shard(n_shards,2)
print(f"dataset_sample size: {len(dataset_sampled)}")
print(dataset_sampled[0])

dataset_sample size: 10
{'instruction': 'Calculate the answer to 66231613267717+60882409809.', 'output': '66231613267717 + 60882409809 = 66292495677526', 'answer': '66292495677526', 'input': '66231613267717 + 60882409809'}


In [8]:
# Tokenize Data

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = 0
# tokenizer.padding_side = "left"  # Allow batched inference

/Users/andrewbeiler/projects/llm-data-driven-optimization/venv_310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [9]:
from random import randint
from itertools import chain
from functools import partial


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_goat_data(sample)}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset_sampled = dataset_sampled.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset_sampled[randint(0, len(dataset_sampled))]["text"])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### Instruction
361034298956504/4

### Answer
361034298956504 / 4 = 90258574739126</s>


In [10]:
print(dataset_sampled[randint(0, len(dataset_sampled))])

{'text': '### Instruction\n8017+5797033\n\n### Answer\n8017 + 5797033 = 5805050</s>'}


In [12]:
# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    batch_chunk_length = 0
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])
    print(batch_total_length)
    print(chunk_length)

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length
    else:
        batch_chunk_length = batch_total_length
    
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset_sampled.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset_sampled.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

Loading cached processed dataset at /Users/andrewbeiler/.cache/huggingface/datasets/tiedong___json/tiedong--goat-55b7467c033a1462/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-554ceb99aab038aa.arrow
Loading cached processed dataset at /Users/andrewbeiler/.cache/huggingface/datasets/tiedong___json/tiedong--goat-55b7467c033a1462/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-adf884cf483af4d2.arrow


Total number of samples: 1


In [13]:
print(type(lm_dataset))
print(lm_dataset[0])

<class 'datasets.arrow_dataset.Dataset'>
{'input_ids': [1, 835, 2799, 4080, 13, 27065, 403, 278, 1234, 304, 29871, 29953, 29953, 29906, 29941, 29896, 29953, 29896, 29941, 29906, 29953, 29955, 29955, 29896, 29955, 29974, 29953, 29900, 29947, 29947, 29906, 29946, 29900, 29929, 29947, 29900, 29929, 29889, 13, 13, 2277, 29937, 673, 13, 29953, 29953, 29906, 29941, 29896, 29953, 29896, 29941, 29906, 29953, 29955, 29955, 29896, 29955, 718, 29871, 29953, 29900, 29947, 29947, 29906, 29946, 29900, 29929, 29947, 29900, 29929, 353, 29871, 29953, 29953, 29906, 29929, 29906, 29946, 29929, 29945, 29953, 29955, 29955, 29945, 29906, 29953, 2, 1, 835, 2799, 4080, 13, 29947, 29900, 29896, 29955, 29974, 29945, 29955, 29929, 29955, 29900, 29941, 29941, 13, 13, 2277, 29937, 673, 13, 29947, 29900, 29896, 29955, 718, 29871, 29945, 29955, 29929, 29955, 29900, 29941, 29941, 353, 29871, 29945, 29947, 29900, 29945, 29900, 29945, 29900, 2, 1, 835, 2799, 4080, 13, 29896, 29929, 29906, 29946, 29953, 29896, 29900, 29

In [14]:
# save train_dataset to s3
ver = "v7"
dataset_name = "goat"
training_input_path = f's3://{sagemaker_session_bucket}/datasets/{dataset_name}/{ver}'
lm_dataset.save_to_disk(training_input_path)

print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

training dataset to: s3://sagemaker-ms-thesis-llm/datasets/goat/v7
